## Clear Seas Research Co-op Position Assessment

The Pacific Pilotage Authority (PPA) is responsible for marine pilotage in the coastal waters of British Columbia. Pilots help vessels, like tankers and cargo ships, navigate through local waters as they come into berth at the Port of Vancouver. The PPA publishes data on vessel movements in their jurisdictional area, which includes information about the type of vessel, the origin and destination of the vessel, and the date and time the pilot was called to the vessel.

Using the information on the PPA’s website, walk us through **how you would calculate the number of tanker visits to the Port of Vancouver in 2023.** The vessel movement data can be found here: https://www.ppa.gc.ca/vessel-movement-data

1. Setup and imports
2. Collecting data
3. Data cleaning and preparation
4. Data analysis

In [31]:
# Import dependencies
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [39]:
# Collecting data
data_2023 = pd.read_excel('All Assignments 2023.xlsx')
data_2023.head(3)

,Vessel,IMO,Call Sign,DWT,GRT,LOA,Beam,S.Draft,Actual Draft,Type,From Location,To Location,First Pilot Ordered,First Pilot Debark,Second Pilot Ordered,Second Pilot Debark
0,SHOFU MARU,9919395,7KMV,100422,58209,235.0,43.00,13.91,7.85,Bulker,PRINCE RUPERT ANCHORAGE 21,TRIGON PACIFIC TERMINALS,2023-01-01 05:00:00,2023-01-01 09:30:00,NaT,NaT
1,ARCHON,9828948,V7HL3,82084,44132,229.0,32.26,14.45,7.60,Bulker,TRIPLE ISLAND,PRINCE RUPERT ANCHORAGE 4,2023-01-01 11:00:00,2023-01-01 16:37:00,NaT,NaT
2,DARYA HEERA,9909077,VRUC9,61083,34534,199.9,32.29,13.00,6.55,Bulker,NANAIMO ANCHORAGE 6,NEPTUNE 2,2023-01-01 18:00:00,2023-01-01 22:50:00,NaT,NaT


- IMO: International Maritime Organization
- DWT: Deadweight Tonnage, the max weight a ship can carry when fully loaded
- GRT: Gross Register Tonnage, a ship's total internal volumne expressed in register tons
- LOA: Length overall, the max length of a vessel's hull measured parallel to the waterline
- Beam: a measure of the width of the ship
- S.Draft: summer draft, the max draft a vessel can have during the summer season
- Actual Draft: the real-time measurement of the vessel's draft at a given moment

In [40]:
data_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13217 entries, 0 to 13216
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Vessel                13217 non-null  object        
 1   IMO                   13217 non-null  int64         
 2   Call Sign             13213 non-null  object        
 3   DWT                   13217 non-null  object        
 4   GRT                   13217 non-null  int64         
 5   LOA                   13217 non-null  float64       
 6   Beam                  13217 non-null  float64       
 7   S.Draft               13193 non-null  float64       
 8   Actual Draft          13217 non-null  float64       
 9   Type                  13217 non-null  object        
 10  From Location         13217 non-null  object        
 11  To Location           13217 non-null  object        
 12  First Pilot Ordered   13217 non-null  datetime64[ns]
 13  First Pilot Deba

In [124]:
# Filter for `Tanker` in the 'Type' column
tanker_2023 = data_2023[(data_2023['Type'] == 'Tanker')]

In [125]:
# Get the name of ports/terminals
to_location = tanker_2023['To Location']
num_categories = to_location.nunique()

# num_categories # 78
unique_locations = to_location.unique().tolist()

# for location in unique_locations:
#     print(location) # doesn't show me all the values

In [126]:
# Define the list of the Port of Vancouver terminal keywords
# Assumption: there is no mislabelled value
port_of_van = ['ANNACIS', 'FRASER SURREY', 'LYNNTERM', 'ALLIANCE', 'CARGILL', 
                'CASCADIA', 'FIBRECO', 'G3 TERMINAL', 'IOCO', 'BC SUGAR',
                'NEPTUNE', 'Pacific Coast Terminals', 'PACIFIC ELEVATOR', 
                'PARKLAND MARINE TERMINAL', 'RICHARDSON', 'SHELLBURN',
                'SUNCOR', 'Univar', 'VANCOUVER WHARVES', 'WESTRIDGE TERMINAL',
                'Westshore Terminals']

# no Richmond Auto, Chemtrade Chemical, Fraser Grain Terminal Ltd., West Coast Reduction.

In [127]:
# create a pattern from the list of keywords
pattern = '|'.join(port_of_van) 

# Filter for rows where 'To Location' contains any of the keywords
tanker_2023 = tanker_2023[tanker_2023['To Location'].str.contains(pattern, case = False, regex = True)]
tanker_2023.head()

,Vessel,IMO,Call Sign,DWT,GRT,LOA,Beam,S.Draft,Actual Draft,Type,From Location,To Location,First Pilot Ordered,First Pilot Debark,Second Pilot Ordered,Second Pilot Debark
7939,NCC NOOR,9399260,HZED,45565,29234,183.00,32.45,12.17,10.20,Tanker,ENGLISH BAY PILOT CHANGE (EBZ),KINDER MORGAN 2 (Vancouver Wharves) (KM2),2023-02-06 13:00:00,2023-02-06 15:10:00,2023-02-06 13:00:00,2023-02-06 15:10:00
7941,NCC NOOR,9399260,HZED,45565,29234,183.00,32.45,12.17,8.80,Tanker,INDIAN ARM ANCHORAGE M (VHM),PARKLAND MARINE TERMINAL (STN),2023-02-10 01:30:00,2023-02-10 03:03:00,NaT,NaT
9610,ARDMORE DEFENDER,9707390,V7HS5,37764,23702,184.06,27.45,11.56,8.50,Tanker,INDIAN ARM ANCHORAGE K (VHK),PORT MOODY 1 (Pacific Coast Terminals) (PM1),2023-02-01 09:00:00,2023-02-01 11:05:00,NaT,NaT
9617,TSURUGI GALAXY,9875501,3FZX7,26143,16589,159.03,27.10,10.12,6.95,Tanker,VANCOUVER HARBOUR ANCHORAGE C (VHC),PORT MOODY 1 (Pacific Coast Terminals) (PM1),2023-02-05 11:30:00,2023-02-05 14:10:00,NaT,NaT
9622,TORM INDIA,9440033,OZPK2,46838,29141,183.17,32.24,12.32,9.00,Tanker,VANCOUVER HARBOUR ANCHORAGE B (VHB),PETRO-CANADA (SUNCOR) (PET),2023-02-07 12:15:00,2023-02-07 15:25:00,NaT,NaT


In [128]:
# Count the number of tanker visits to Port of Vancouver
tanker_visits_count_2023 = tanker_2023.shape[0]
tanker_visits_count_2023

230

https://www.portvancouver.com/cargo-and-terminals

In [32]:
# Collecting data (2016 - 2024)
data_2016 = pd.read_excel('All Assignments 2016 for web.xls')
data_2016.head(3)

,Vessel,IMO,Call Sign,DWT,GRT,LOA,Beam,S.Draft,Actual Draft,Type,From Location,To Location,First Pilot Ordered,First Pilot Debark,Second Pilot Ordered,Second Pilot Debark
0,STAR FUJI,8309830,LAVX4,40850,25345,187.31,29.47,11.82,9.62,General Cargo,KITIMAT EUROCAN 1,TRIPLE ISLAND,2016-01-01 00:30:00,2016-01-01 10:25:00,2016-01-01 00:30:00,2016-01-01 10:25:00
1,ZAMPA BLUE,9454163,3FYS6,178459,92249,292.00,44.98,17.99,12.57,Bulk Carrier,ROBERTS BANK 1,BROTCHIE (SEA),2016-01-01 01:00:00,2016-01-01 08:33:00,NaT,NaT
2,GIULIA I,9638135,9HA3585,39202,25303,179.99,30.00,10.5,7.35,Bulk Carrier,TRIPLE ISLAND,PRINCE RUPERT ANCHORAGE 5,2016-01-01 02:00:00,2016-01-01 05:00:00,NaT,NaT


In [33]:
data_2017 = pd.read_excel('All Assignments 2017 for web.xls')
data_2017.head(3)

,Vessel,IMO,Call Sign,DWT,GRT,LOA,Beam,S.Draft,Actual Draft,Type,From Location,To Location,First Pilot Ordered,First Pilot Debark,Second Pilot Ordered,Second Pilot Debark
0,ZAO GALAXY,9566150,V7YK9,26198,16408,159.03,27.13,10.01,10.05,Tanker,BROTCHIE (SEA),LYNN TERM 7,2017-01-01 02:00:00,2017-01-01 08:46:00,NaT,NaT
1,YOUNGHEUNG,9502934,3EUB4,179221,93152,291.97,45.00,18.22,9.52,Bulk Carrier,ROYAL ROADS ANCHORAGE D,ROBERTS BANK 1,2017-01-01 03:00:00,2017-01-01 08:55:00,NaT,NaT
2,NECKLACE,9548225,5BSN3,92903,51208,229.20,38.04,14.9,6.94,Bulk Carrier,BROTCHIE (SEA),ENGLISH BAY ANCHORAGE 01,2017-01-01 03:00:00,2017-01-01 09:17:00,NaT,NaT


In [34]:
data_2018 = pd.read_excel('All Assignments 2018 for web.xls')
data_2018.head(3)

,Vessel,IMO,Call Sign,DWT,GRT,LOA,Beam,S.Draft,Actual Draft,Type,From Location,To Location,First Pilot Ordered,First Pilot Debark,Second Pilot Ordered,Second Pilot Debark
0,IVS NORTH BERWICK,9740902,9V3774,60475,34233,199.98,32.29,12.96,12.70,Bulk Carrier,ENGLISH BAY ANCHORAGE 15,BROTCHIE (SEA),2018-01-01 01:00:00,2018-01-01 07:48:00,NaT,NaT
1,SAKURA OCEAN,9599822,3FRC8,38239,23268,179.97,29.80,10.54,10.45,Bulk Carrier,PORT MOODY 2,BROTCHIE (SEA),2018-01-01 04:00:00,2018-01-01 12:16:00,2018-01-01 04:00:00,2018-01-01 12:16:00
2,ANDREAS K,9438121,5BVX2,91873,51130,229.50,36.97,14.7,9.50,Bulk Carrier,NANAIMO ANCHORAGE 4,VANCOUVER HARBOUR ANCHORAGE A,2018-01-01 06:00:00,2018-01-01 10:00:00,NaT,NaT


In [35]:
data_2019 = pd.read_excel('All Assignments 2019.xls')
data_2019.head(3)

,Vessel,IMO,Call Sign,DWT,GRT,Beam,LOA,S.Draft,Actual Draft,Type,From Location,To Location,First Pilot Ordered,First Pilot Debark,Second Pilot Ordered,Second Pilot Debark
0,OCEAN CENTURY,9485033,TCZY7,55848,31540,32.30,189.99,12.74,6.95,Bulk Carrier,PRINCE RUPERT ANCHORAGE 7,STEWART ORE DOCK (Bulk Terminals),2019-01-01 02:00:00,2019-01-01 13:00:00,2019-01-01 02:00:00,2019-01-01 13:00:00
1,MISATO K,9634438,3FLD9,56048,31754,32.25,189.99,12.72,6.81,Bulk Carrier,BROTCHIE (SEA),VANCOUVER HARBOUR ANCHORAGE C,2019-01-01 04:00:00,2019-01-01 10:40:00,NaT,NaT
2,DARYA MOTI,9471252,VRGN2,80545,43842,32.29,229.00,14.45,6.60,Bulk Carrier,BROTCHIE (SEA),ENGLISH BAY ANCHORAGE 05,2019-01-01 05:00:00,2019-01-01 11:00:00,NaT,NaT


In [36]:
data_2020 = pd.read_excel('All Assignments 2020.xlsx')
data_2020.head(3)

,Vessel,IMO,Call Sign,DWT,GRT,Beam,LOA,S.Draft,Actual Draft,Type,From Location,To Location,First Pilot Ordered,First Pilot Debark,Second Pilot Ordered,Second Pilot Debark
0,SAGAMI,9379911,3ETH,33614,19420,26.63,170.00,11.32,8.50,Tanker,PORT MOODY 1 (Pacific Coast Terminals),VANCOUVER HARBOUR ANCHORAGE N,2020-01-01 02:00:00,2020-01-01 03:10:00,NaT,NaT
1,NAVIG8 SPARK,9731743,V7ST4,25196,16084,25.00,160.00,10.37,5.50,Tanker,ENGLISH BAY ANCHORAGE 11,PORT MOODY 1 (Pacific Coast Terminals),2020-01-01 02:00:00,2020-01-01 05:40:00,NaT,NaT
2,SWIFT ACE,9338838,C6XB8,18865,58685,32.26,199.95,9.82,9.05,Car Carrier,BROTCHIE (SEA),ENGLISH BAY ANCHORAGE 11,2020-01-01 04:00:00,2020-01-01 08:50:00,NaT,NaT


In [37]:
data_2021_Q4 = pd.read_excel('All Assignments Q4 2021.xlsx')
data_2021_Q4.head(3)

,Vessel,IMO,Call Sign,DWT,GRT,LOA,Beam,S.Draft,Actual Draft,Type,From Location,To Location,First Pilot Ordered,First Pilot Debark,Second Pilot Ordered,Second Pilot Debark
0,DORIC WARRIOR,9453511,A8WQ4,93115,51208,230.00,38.06,14.9,6.99,Bulk Carrier,BROTCHIE (SEA),VANCOUVER HARBOUR ANCHORAGE C,2021-01-01 05:00:00,2021-01-01 11:50:00,NaT,NaT
1,RANGAKU,9866627,D5WF2,63533,35645,199.98,32.24,13.42,9.17,Bulk Carrier,VANCOUVER HARBOUR ANCHORAGE E,PLUMPER SOUND ANCHORAGE C,2021-01-01 07:00:00,2021-01-01 12:15:00,NaT,NaT
2,KEN UN,9727089,3EGQ5,37429,22469,179.99,30.00,10.33,6.62,Bulk Carrier,BROTCHIE (SEA),KULLEET BAY ANCHORAGE 2,2021-01-01 08:00:00,2021-01-01 12:55:00,NaT,NaT


In [38]:
data_2022 = pd.read_excel('All Assignments 2022.xls')
data_2022.head(3)

,Vessel,IMO,Call Sign,DWT,GRT,LOA,Beam,S.Draft,Actual Draft,Type,From Location,To Location,First Pilot Ordered,First Pilot Debark,Second Pilot Ordered,Second Pilot Debark
0,APL TURKEY,9532771,A8TR2,72912,71787,293.20,40.0,14.02,13.50,Container,PRINCE RUPERT ANCHORAGE 24,PRINCE RUPERT ANCHORAGE 29,2022-01-01 02:30:00,2022-01-01 02:25:00,NaT,NaT
1,NORDIC CALLAO,9796822,3ECB4,26029,16572,159.03,27.1,10.12,9.95,Tanker,PORT MOODY 1 (Pacific Coast Terminals),ENGLISH BAY PILOT CHANGE,2022-01-01 03:50:00,2022-01-01 06:50:00,NaT,NaT
2,SYNERGY BUSAN,9450571,V7A2725,50727,40030,260.00,32.3,12.60,9.70,Container,VANCOUVER HARBOUR ANCHORAGE B,BROTCHIE (SEA),2022-01-01 05:00:00,2022-01-01 09:55:00,NaT,NaT


In [23]:
data_2024_Q1 = pd.read_excel('2024 Q1 All Assignments.xlsx')
data_2024_Q1.head(3)

,Vessel,IMO,Call Sign,DWT,GRT,LOA,Beam,S.Draft,Actual Draft,Type,From Location,To Location,First Pilot Ordered,First Pilot Debark,Second Pilot Ordered,Second Pilot Debark
0,UNITY VENTURE,9773739,VRPW9,112186,60264,237.00,44.0,15.00,8.2,Tanker,INDIAN ARM ANCHORAGE L (VHL),WESTRIDGE TERMINAL BERTH 1 (WR1),2024-01-01 06:00:00,2024-01-01 08:50:00,NaT,NaT
1,AURORA,9920966,WDL8377,393,782,124.00,14.0,3.66,3.4,Tug,INDIAN ARM ANCHORAGE N (VHN),IOCO (IMPERIAL OIL CO.) (IOC),2024-01-01 06:00:00,2024-01-01 08:30:00,NaT,NaT
2,MSC ATHOS,9618317,5LKV7,116993,96262,299.95,48.2,15.00,10.4,Container,BROTCHIE (SEA) (BRO),VANTERM 6 (VT6),2024-01-01 07:00:00,2024-01-01 12:57:00,NaT,NaT


In [24]:
data_2024_Q2 = pd.read_excel('2024 Q2 All Assignments.xlsx')
data_2024_Q2.head(3)

,Vessel,IMO,Call Sign,DWT,GRT,LOA,Beam,S.Draft,Actual Draft,Type,From Location,To Location,First Pilot Ordered,First Pilot Debark,Second Pilot Ordered,Second Pilot Debark
0,FEDERAL OAK,9860415,D5WH9,60385,34582,199.99,32.25,12.95,8.79,Bulker,ENGLISH BAY ANCHORAGE 10 (E10),SANDHEADS (SHD),2024-04-01 01:00:00,2024-04-01 02:58:00,NaT,NaT
1,COSCO BOSTON,9335173,3ELF2,68240,54778,294.07,32.21,13.50,10.70,Container,TRIPLE ISLAND (TPL),PRINCE RUPERT FAIRVIEW 2 (South) (PF2),2024-04-01 03:00:00,2024-04-01 06:58:00,NaT,NaT
2,FEDERAL OAK,9860415,D5WH9,60385,34582,199.99,32.25,12.95,8.79,Bulker,SANDHEADS (SHD),FRASER SURREY 4 (FS4),2024-04-01 03:00:00,2024-04-01 06:35:00,NaT,NaT


In [25]:
data_2024_Q3 = pd.read_excel('2024 Q3 All Assignments.xlsx')
data_2024_Q3.head(3)

,Vessel,IMO,Call Sign,DWT,GRT,LOA,Beam,S.Draft,Actual Draft,Type,From Location,To Location,First Pilot Ordered,First Pilot Debark,Second Pilot Ordered,Second Pilot Debark
0,RUBY PRINCESS,9378462,ZCDY2,8044,113561,288.60,36.02,8.5,8.5,Passenger,PINE ISLAND (PIN),CANADA PLACE EAST (CPE),2024-07-01 12:30:00,2024-07-02 07:30:00,2024-07-01 12:30:00,2024-07-02 07:30:00
1,ROALD AMUNDSEN,9813072,LAZP7,1800,21765,140.00,23.60,5.5,5.5,Passenger,NORTHLAND CRUISE TERMINAL (NCT),CANADA PLACE WEST (CPW),2024-07-01 23:45:00,2024-07-04 07:45:00,2024-07-01 23:45:00,2024-07-04 07:45:00
2,ZAANDAM,9156527,PDAN,6150,61396,237.95,32.25,8.1,8.1,Passenger,TRIPLE ISLAND (TPL),CANADA PLACE WEST (CPW),2024-07-02 00:15:00,2024-07-03 07:30:00,2024-07-02 00:15:00,2024-07-03 07:30:00


### Data cleaning and preparation

In [26]:
data_2016.describe()

,GRT,LOA,Beam,Actual Draft,First Pilot Ordered,First Pilot Debark,Second Pilot Ordered,Second Pilot Debark
count,12646.000000,12646.000000,12646.000000,12646.000000,12646,12646,720,720
mean,44646.862012,216.977517,32.442702,9.280378,2016-07-06 07:54:12.952712448,2016-07-06 13:05:09.584058368,2016-07-11 09:40:20.166666752,2016-07-12 03:41:19.666666496
min,183.000000,28.960000,7.930000,2.130000,2016-01-01 00:30:00,2016-01-01 05:00:00,2016-01-01 00:30:00,2016-01-01 10:25:00
25%,25569.000000,183.000000,30.040000,7.460000,2016-04-08 05:07:30,2016-04-08 08:48:00,2016-05-23 16:15:00,2016-05-24 07:55:45
50%,38606.000000,199.980000,32.260000,8.800000,2016-07-07 12:45:00,2016-07-07 16:37:30,2016-07-11 20:00:00,2016-07-12 18:38:30
75%,55600.000000,234.980000,32.300000,11.050000,2016-10-02 18:11:15,2016-10-02 23:20:00,2016-08-30 02:52:30,2016-08-30 23:46:45
max,141823.000000,366.500000,50.050000,18.760000,2016-12-31 22:00:00,2017-01-01 07:30:00,2016-12-31 22:00:00,2017-01-01 07:30:00
std,26534.760874,55.123921,5.917003,2.372466,NaN,NaN,NaN,NaN


In [30]:
data_2016.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12646 entries, 0 to 12645
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Vessel                12646 non-null  object        
 1   IMO                   12646 non-null  object        
 2   Call Sign             12646 non-null  object        
 3   DWT                   12500 non-null  object        
 4   GRT                   12646 non-null  int64         
 5   LOA                   12646 non-null  float64       
 6   Beam                  12646 non-null  float64       
 7   S.Draft               12600 non-null  object        
 8   Actual Draft          12646 non-null  float64       
 9   Type                  12646 non-null  object        
 10  From Location         12646 non-null  object        
 11  To Location           12646 non-null  object        
 12  First Pilot Ordered   12646 non-null  datetime64[ns]
 13  First Pilot Deba